In [261]:
from models import Statements_table_log
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from config import DATABASE_URI
from scrapping_sources.Macrotrend import Macrotrend
import datetime
import pandas as pd

In [262]:
USERNAME='postgres'
HOST='database-webscrap.ccaaerr6cq44.eu-west-3.rds.amazonaws.com'
DATABASE_NAME='webscrapping'
PASSWORD='postgres5678'
PORT=5432
DATABASE_URI = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}"\

session = sessionmaker(bind=create_engine(DATABASE_URI))
s = session()
M = Macrotrend()

In [263]:
import pandas as pd
import requests, json, re
from bs4 import BeautifulSoup as bs
import datetime
import concurrent.futures
from itertools import repeat

class Macrotrend():
        
    def get_statement(self, ticker, statement, time_format):

        statement_data = None #default load

        if time_format == 'annual':

            try:
                r = requests.get('https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/' + statement)
                print(ticker, r)
                p = re.compile(r'var originalData = (.*);')
                p2 = re.compile(r'datafields:[\s\S]+(\[[\s\S]+?\]),')
                p3 = re.compile(r'\d{4}-\d{2}-\d{2}')
                data = json.loads(p.findall(r.text)[0])
                s = re.sub('\r|\n|\t|\s','',p2.findall(r.text)[0])
                fields = p3.findall(s)
                fields.insert(0, 'field_name') # only headers of interest.
                results = []

                for item in data: #loop initial list of dictionaries
                    row = {}
                    for f in fields: #loop keys of interest to extract from current dictionary
                        if f == 'field_name':  #this is an html value field so needs re-parsing
                            soup2 = bs(item[f],'lxml')
                            row[f] = soup2.select_one('a,span').text
                        else:
                            row[f] = item[f]
                    results.append(row)

                # return pd.DataFrame(results, columns = fields)
                statement_data = pd.DataFrame(results, columns = fields)

                return statement_data
        
            except:
                print(f"No data available for {ticker}, {statement}, annual")
                pass
            # pass
        else:

            try:

                r = requests.get(f'https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/' + statement)
                company_name = r.url.split('/')[6]
                r = requests.get(f'https://www.macrotrends.net/stocks/charts/'+ ticker + '/' + company_name + '/' + statement + '?freq=Q')
                print(ticker, r)
                p = re.compile(r'var originalData = (.*);')
                p2 = re.compile(r'datafields:[\s\S]+(\[[\s\S]+?\]),')
                p3 = re.compile(r'\d{4}-\d{2}-\d{2}')
                data = json.loads(p.findall(r.text)[0])
                s = re.sub('\r|\n|\t|\s','',p2.findall(r.text)[0])
                fields = p3.findall(s)
                fields.insert(0, 'field_name') # only headers of interest.
                results = []

                for item in data: #loop initial list of dictionaries
                    row = {}
                    for f in fields: #loop keys of interest to extract from current dictionary
                        if f == 'field_name':  #this is an html value field so needs re-parsing
                            soup2 = bs(item[f],'lxml')
                            row[f] = soup2.select_one('a,span').text
                        else:
                            row[f] = item[f]
                    results.append(row)
                statement_data = pd.DataFrame(results, columns = fields)

                return statement_data

            except:

                print(f"No data available for {ticker}, {statement}, quarterly")

                pass

    def move_column(self, df, column, pos):

        col = df[column]
        df.drop(columns=[column],inplace = True)
        df.insert(pos, column, col)

        return df
    
    def generate_statement_key(self, statement, time_format):

        report_formats  = {'quarterly'  : 'Q',
                            'annual'    : 'A'}
        statements      = {'income-statement'       :'IS',
                            'balance-sheet'         :'BS',
                            'cash-flow-statement'   :'CF',
                            'financial-ratios'      :'R'}

        statement_key = statements[statement] + '-' + report_formats[time_format]

        return statement_key

    def arrange_data(self, ticker, statement, time_format):

        data_dict = []
        report_formats = {'quarterly' : 'Q',
                        'annual': 'A'}
        statements = {'income-statement':'IS',
                    'balance-sheet':'BS',
                    'cash-flow-statement':'CF',
                    'financial-ratios':'R'}

        df = self.get_statement(ticker, statement, time_format)

        if isinstance(df, pd.DataFrame):

            for i in df.columns[1:]:
                keys = df[df.columns[0]]
                data = df[i].values
                data = {k:v for k,v in zip(keys, data)}
                date = i
                data['date'] = date
                data['ticker'] = ticker
                data['statement_format'] = statements[statement] + '-' + report_formats[time_format]
                data_dict.append(data)

            df = pd.DataFrame(data_dict)
            df = self.move_column(df, 'date', 0)
            df = self.move_column(df, 'ticker', 1)
            df = self.move_column(df, 'statement_format', 2)

            my_series = []
            line_item = []
            amount = []
            date = []
            ticker = []
            security_id = []
            statement_format = []

            for i in df.iterrows():

                serie = pd.Series(i)[1]
                my_series.append(serie)
                [statement_format.append(serie[2]) for i in range(len(serie.index.values[4:]))]
                # [security_id.append(serie[3]) for i in range(len(serie.index.values[4:]))]
                [line_item.append(i) for i in  serie.index.values[4:]]
                [amount.append(i) for i in serie.values[4:]]
                [date.append(serie[0]) for i in  range(len(serie.index.values[4:]))]
                [ticker.append(serie[1]) for i in range(len(serie.index.values[4:]))]

            data = pd.DataFrame([date, statement_format, ticker, security_id,line_item, amount]).T
            data.columns = ['date','statement','ticker','security_id','line_item','amount']

            return data

        else:
            pass

    def generate_statement_table_multi(self, ticker_list, statement, time_format):

        table   = []

        for i in ticker_list:

            df = self.arrange_data(i, statement, time_format)

            if isinstance(df, pd.DataFrame):
                table.append(df)
            else:
                pass

        table_concat = pd.concat(table)
        table_concat['amount'] =  pd.to_numeric(table_concat['amount'])
        return table_concat

    def generate_statement_table_multi_threading(self, ticker_list, statement, time_format):

        table = []

        def create_table(ticker, statement, time_format):

            df = self.arrange_data(ticker, statement, time_format)
            if isinstance(df, pd.DataFrame):
                table.append(df)

        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            executor.map(create_table, ticker_list, repeat(statement), repeat(time_format))

        table_concat = pd.concat(table)
        table_concat['amount'] =  pd.to_numeric(table_concat['amount'])

        return table_concat
    
    def latest_ending_period_available(self, ticker, statement='balance-sheet', time_format='quarterly'):

        try:
            df = self.get_statement(ticker, statement, time_format)
            raw_date = df.columns[1]
            converted_date = datetime.datetime.strptime(str(raw_date), '%Y-%m-%d').strftime("%Y-%m")
        except:
            converted_date = None

        return converted_date
    

In [264]:
from models import Security
engine = create_engine(DATABASE_URI)
Session = sessionmaker(bind=engine)

s = Session()
r = s.query(Security.ticker, Security.id).all()
sec_id_map = {i[0]:i[1] for i in r}
# df['security_id'] = df.ticker.map(sec_id_map)
# df[df['date','ticker', 'statement']]
# sec_id_map

In [265]:
def generate_statement_list(ticker, statement, time_format):

    df = M.arrange_data(ticker, statement, time_format)
    df = df[['date','ticker', 'statement', 'security_id']].drop_duplicates()
    df["security_id"] = df["ticker"].map(sec_id_map)
    # Getting there, but what's next?...
    # 1 - Create a statement_id
    # 
    return df

In [277]:
generate_statement_list('AAPL','income-statement', 'annual')

AAPL <Response [200]>


,date,ticker,statement,security_id
0,2020-09-30,AAPL,BS-A,17
22,2019-09-30,AAPL,BS-A,17
44,2018-09-30,AAPL,BS-A,17
66,2017-09-30,AAPL,BS-A,17
88,2016-09-30,AAPL,BS-A,17
110,2015-09-30,AAPL,BS-A,17
132,2014-09-30,AAPL,BS-A,17
154,2013-09-30,AAPL,BS-A,17
176,2012-09-30,AAPL,BS-A,17
198,2011-09-30,AAPL,BS-A,17


In [267]:
r = requests.get('https://www.macrotrends.net/stocks/charts/' + 'AAPL' + '/' + 'AAPL' + '/' + 'income-statement')
p = re.compile(r'var originalData = (.*);')
p2 = re.compile(r'datafields:[\s\S]+(\[[\s\S]+?\]),')
p3 = re.compile(r'\d{4}-\d{2}-\d{2}')
data = json.loads(p.findall(r.text)[0])
s = re.sub('\r|\n|\t|\s','',p2.findall(r.text)[0])
fields = p3.findall(s)
# fields.insert(0, 'field_name') # only headers of interest.
results = []
# p.findall(r.text)[0]

In [268]:
def item_amount_unit(ticker, statement, time_format):

    if time_format == "annual":

        r = requests.get('https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/' + statement)
        text = r.text
        pattern = re.compile(r"var columnList = (.*);", re.DOTALL)
        matches = pattern.findall(text)
        matches = re.sub("\r|\n|\t",'',  matches[0])
        pattern2 = re.compile(r"'(.*?)'")
        match = pattern2.findall(str(matches))[0]

        return str(match).split(" | ")[1]

    elif time_format == "quarterly":

        r = requests.get(f'https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/' + statement)
        company_name = r.url.split('/')[6]
        r = requests.get(f'https://www.macrotrends.net/stocks/charts/'+ ticker + '/' + company_name + '/' + statement + '?freq=Q')
        text = r.text
        pattern = re.compile(r"var columnList = (.*);", re.DOTALL)
        matches = pattern.findall(text)
        matches = re.sub("\r|\n|\t",'',  matches[0])
        pattern2 = re.compile(r"'(.*?)'")
        match = pattern2.findall(str(matches))[0]

        return str(match).split(" | ")[1]



In [274]:
item_amount_unit('MTH', "income-statement", "quarterly")

'Millions of US $ except per share data'

In [270]:
for item in data: #loop initial list of dictionaries
    row = {}
    for f in fields: #loop keys of interest to extract from current dictionary
        if f == 'field_name':  #this is an html value field so needs re-parsing
            soup2 = bs(item[f],'lxml')
            row[f] = soup2.select_one('a,span').text
        else:
            row[f] = item[f]
    results.append(row)

In [271]:
fields

['2020-09-30',
 '2019-09-30',
 '2018-09-30',
 '2017-09-30',
 '2016-09-30',
 '2015-09-30',
 '2014-09-30',
 '2013-09-30',
 '2012-09-30',
 '2011-09-30',
 '2010-09-30',
 '2009-09-30',
 '2008-09-30',
 '2007-09-30',
 '2006-09-30',
 '2005-09-30']

In [272]:
p2

re.compile(r'datafields:[\s\S]+(\[[\s\S]+?\]),', re.UNICODE)

In [238]:
test_text = "Hello, My name is Paul. I am trying out Python; regular expressions"

pattern = re.compile(r"[*]")
pattern.findall(test_text)

[]

In [15]:
def no_table_log(ticker, stmnt, t_format):

    r = s.query(Statements_table_log.__table__).filter(
        Statements_table_log.ticker == ticker,
        Statements_table_log.statement == stmnt,
        Statements_table_log.time_format == t_format,
        Statements_table_log.period == M.latest_ending_period_available(ticker)
    ).all()

    return len(r) == 0

In [16]:
no_table_log("TAYD", 'income-statement', 'annual')

TAYD <Response [200]>


False